<a href="https://colab.research.google.com/github/jus-tinian/vix_calc/blob/master/SparkTD_SPXquotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create Working Env

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [3]:
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 217.8MB 61kB/s 
     |████████████████████████████████| 204kB 30.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=96cdc6c09ef69ebc9a92df6a5ddfafaf49b5f230f189f8c7f5d1e323b5e61919
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Imports

In [0]:
import requests
import datetime

import pyspark
import pandas as pd

from pyspark.sql import SparkSession

from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType, DateType
from pyspark.sql import DataFrame

from functools import reduce

## Create Spark Session

In [0]:
APP_NAME = "VIXcalc"
SPARK_URL = "local[*]"

In [0]:
spark = SparkSession.builder.master(SPARK_URL).appName(APP_NAME).getOrCreate()

In [8]:
spark

## Make Dates

In [0]:
today = datetime.datetime.today().date()

In [0]:
farDate = today + datetime.timedelta(days=37)

In [0]:
nearDate = today + datetime.timedelta(days=23)

## Get API response

In [0]:
# your key here
KEY = 'your_key_here'

In [0]:
url = f"https://api.tdameritrade.com/v1/marketdata/chains?apikey={KEY}&symbol=%24SPX.X&includeQuotes=TRUE&strategy=SINGLE&range=ALL&fromDate={str(nearDate)}&toDate={str(farDate)}"

In [0]:
req = requests.get(url)

In [0]:
reqJSON = req.json()

## Set Keys to access reqJSON

In [0]:
putKey = 'putExpDateMap'

In [0]:
callKey = 'callExpDateMap'

In [0]:
dateKeys = [i for i in list(req.json()[putKey].keys()) if pd.to_datetime(i[:-3]).dayofweek == 4]

In [0]:
nearKey = min(dateKeys)
farKey = max(dateKeys)
nearExpiration = datetime.datetime.strptime(nearKey[:-3], '%Y-%m-%d').date()
farExpiration = datetime.datetime.strptime(farKey[:-3], '%Y-%m-%d').date()

## Set Interest Rate

In [0]:
RATE = reqJSON['interestRate']

## Set Columns to Keep

In [0]:
minKeepCols = ['putCall', 'symbol', 'bid', 'ask', 'expirationDate', 'daysToExpiration', 'inTheMoney', 'strikePrice']

## UDFs

In [0]:
convertEpochToDate = udf(lambda msEpoch: datetime.datetime.fromtimestamp(msEpoch/1000).date(), DateType())

In [0]:
convertEpochToTimeStamp = udf(lambda msEpoch: datetime.datetime.fromtimestamp(msEpoch/1000), TimestampType())

## Prep Pipe

In [0]:
def from_json(jsonObj, sideKey, termKey, minKeepColsParam=minKeepCols):

  minStrikeKey = list(reqJSON[sideKey][termKey].keys())[0]

  strikesList = list(jsonObj[sideKey][termKey].keys())[1:]

  source_df = spark.createDataFrame(pd.DataFrame(jsonObj[sideKey][termKey][minStrikeKey])[minKeepCols])

  dfs = [source_df] + [spark.createDataFrame(pd.DataFrame(jsonObj[sideKey][termKey][v])[minKeepColsParam]) for i, v in enumerate(strikesList)]

  source_df1 = reduce(DataFrame.union, dfs)

  source_df2 = source_df1.where('bid != 0')

  nonWeeklyCount = source_df2.filter(~source_df1.symbol.contains('W')).count()

  if nonWeeklyCount != 0:
    source_df2 = source_df2.filter(~source_df2.symbol.contains('W'))

  source_df3 = source_df2.withColumn(
      'expiration',
      convertEpochToDate('expirationDate')).select(
          'strikePrice',
          'putCall',
          'symbol',
          'bid',
          'ask',
          'daysToExpiration',
          'inTheMoney',
          'expiration')
      
  return source_df3

# Near Puts

In [0]:
nearPuts = from_json(reqJSON, putKey, nearKey)

In [27]:
nearPuts.show(5)

+-----------+-------+----------------+----+----+----------------+----------+----------+
|strikePrice|putCall|          symbol| bid| ask|daysToExpiration|inTheMoney|expiration|
+-----------+-------+----------------+----+----+----------------+----------+----------+
|     1150.0|    PUT|SPXW_061220P1150|0.05| 0.2|              30|     false|2020-06-12|
|     1250.0|    PUT|SPXW_061220P1250| 0.1| 0.3|              30|     false|2020-06-12|
|     1300.0|    PUT|SPXW_061220P1300| 0.1| 0.3|              30|     false|2020-06-12|
|     1350.0|    PUT|SPXW_061220P1350|0.15|0.35|              30|     false|2020-06-12|
|     1400.0|    PUT|SPXW_061220P1400| 0.2|0.45|              30|     false|2020-06-12|
+-----------+-------+----------------+----+----+----------------+----------+----------+
only showing top 5 rows



## Far Puts

In [0]:
farPuts = from_json(reqJSON, putKey, farKey)

In [29]:
farPuts.show(5)

+-----------+-------+---------------+----+----+----------------+----------+----------+
|strikePrice|putCall|         symbol| bid| ask|daysToExpiration|inTheMoney|expiration|
+-----------+-------+---------------+----+----+----------------+----------+----------+
|      900.0|    PUT| SPX_061920P900|0.05| 0.1|              37|     false|2020-06-19|
|     1000.0|    PUT|SPX_061920P1000| 0.1|0.15|              37|     false|2020-06-19|
|     1100.0|    PUT|SPX_061920P1100| 0.1| 0.2|              37|     false|2020-06-19|
|     1150.0|    PUT|SPX_061920P1150| 0.1| 0.2|              37|     false|2020-06-19|
|     1200.0|    PUT|SPX_061920P1200|0.15| 0.3|              37|     false|2020-06-19|
+-----------+-------+---------------+----+----+----------------+----------+----------+
only showing top 5 rows



# Near Calls

In [0]:
nearCalls = from_json(reqJSON, callKey, nearKey)

In [31]:
nearCalls.show(5)

+-----------+-------+---------------+------+------+----------------+----------+----------+
|strikePrice|putCall|         symbol|   bid|   ask|daysToExpiration|inTheMoney|expiration|
+-----------+-------+---------------+------+------+----------------+----------+----------+
|      500.0|   CALL|SPXW_061220C500|2309.5|2315.6|              30|      true|2020-06-12|
|      600.0|   CALL|SPXW_061220C600|2209.6|2215.6|              30|      true|2020-06-12|
|      700.0|   CALL|SPXW_061220C700|2109.3|2115.6|              30|      true|2020-06-12|
|      800.0|   CALL|SPXW_061220C800|2009.7|2015.7|              30|      true|2020-06-12|
|      900.0|   CALL|SPXW_061220C900|1909.4|1915.8|              30|      true|2020-06-12|
+-----------+-------+---------------+------+------+----------------+----------+----------+
only showing top 5 rows



## Far Calls

In [0]:
farCalls = from_json(reqJSON, callKey, farKey)

In [33]:
farCalls.show(5)

+-----------+-------+--------------+------+------+----------------+----------+----------+
|strikePrice|putCall|        symbol|   bid|   ask|daysToExpiration|inTheMoney|expiration|
+-----------+-------+--------------+------+------+----------------+----------+----------+
|      100.0|   CALL|SPX_061920C100|2704.2|2720.4|              37|      true|2020-06-19|
|      200.0|   CALL|SPX_061920C200|2604.3|2620.5|              37|      true|2020-06-19|
|      300.0|   CALL|SPX_061920C300|2504.4|2520.0|              37|      true|2020-06-19|
|      400.0|   CALL|SPX_061920C400|2404.5|2420.1|              37|      true|2020-06-19|
|      500.0|   CALL|SPX_061920C500|2304.5|2320.1|              37|      true|2020-06-19|
+-----------+-------+--------------+------+------+----------------+----------+----------+
only showing top 5 rows

